### Testing

In [1]:
# Import the necessary libraries

import requests
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Fetching the URL for Premier League standings
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [ ]:
# Sending a request to retrieve data from the specified URL
data = requests.get(standings_url)

In [ ]:
# Creating a BeautifulSoup object to parse the HTML content of the fetched data
soup = BeautifulSoup(data.text)

In [ ]:
# Selecting the first table with the class 'stats_table' from the parsed HTML content
standings_table = soup.select('table.stats_table')[0]

In [ ]:
# Finding all anchor tags ('a') within the selected standings table
links = standings_table.find_all('a')

In [ ]:
# Extracting the 'href' attribute from each anchor tag in the standings table and storing them in a list
links = [l.get('href') for l in links]

In [ ]:
# Filtering the list of links to only include those related to team squads ('/squads').
links = [l for l in links if '/squads' in l]

In [ ]:
# Constructing complete URLs for each team's squad page by appending the relative links to the base URL.
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
# Selecting the URL of the first team's squad page from the constructed list.
team_url = team_urls[0]

In [ ]:
# Sending a request to retrieve data from the specified team's squad page URL.
data = requests.get(team_url)

In [ ]:
# Parsing HTML tables from the fetched data, specifically targeting tables containing information about scores and fixtures.
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [2]:
# Displaying the first few rows of the extracted table containing scores and fixtures.
matches[0].head()

NameError: name 'matches' is not defined

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get('href') for l in links]

In [ ]:
# Filtering the list of links to include only those with a valid value and containing the specified subpath ('all_comps/shooting/').
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
# Parsing HTML tables from the fetched data, specifically targeting the table related to shooting statistics.
shooting = pd.read_html(data.text, match='Shooting')[0]

In [ ]:
shooting.head(5)

In [ ]:
# Dropping one level from the multi-level column index in the shooting statistics DataFrame.
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head(5)

In [ ]:
# Merging the matches and shooting statistics DataFrames on the 'Date' column to consolidate relevant information for further analysis.
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

### Creating CSV File from Premier League Stats website from 2021 to 2023 season

In [3]:
# Creating a list of years (in reverse order) from 2024 to 2021.
years = list(range(2024, 2021, -1))

In [4]:
# Initializing an empty list to store match data.
all_matches = []

In [5]:
# Fetching the URL for Premier League standings.
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [6]:
import time
for year in years:
    # Fetching standings data for the current year
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    # Extracting team URLs from standings data
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # Updating standings URL for the previous season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    # Looping through each team's URL
    for team_url in team_urls:
        # Extracting team name from URL
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        # Fetching data from the team's squad page
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        # Extracting shooting statistics URL from the team's squad page
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        # Fetching shooting statistics data
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            # Merging match and shooting data on the 'Date' column
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
            
        # Filtering data for Premier League matches
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        # Adding 'Season' and 'Team' columns
        team_data["Season"] = year
        team_data["Team"] = team_name
        
        # Appending team data to the list
        all_matches.append(team_data)
        
        # Adding a delay to avoid overloading the server
        time.sleep(5)

In [7]:
# Concatenating all team match data into a single DataFrame (match_df)
match_df = pd.concat(all_matches)

In [8]:
# Converting column names to lowercase.
match_df.columns = [c.lower() for c in match_df.columns]

In [9]:
# Saving the match_df DataFrame to a CSV file.
match_df.to_csv("matches.csv")